# 1. Getting Started

multiTFA is a python package for accurate thermodynamic flux analysis. multiTFA is built on COBRApy to take advantage of COBRA model structure. For installation instructions please refer to **INSTALL.rst**.

## 1.1. Building a multiTFA model

multiTFA models can be readily constructed from COBRA model objects. To create a model, we require 

1. A COBRA model

2. Metabolite KEGG/SEED identifiers - This is required to match and retrieve thermodyanamic information against our database. Metabolite identifiers can be added to the multiTFA model later. For simplicity, we use the same nomneclature as `equilibrator-api`. 

3. Compartment specific pH and ionic strength information. To calculate Gibbs energy related to transmembrane transport, information about membrane potential is also required in the form of pandas Dataframes. 

Additionally, for accurate thermodynamic calculations, intracellular metabolite concentrations can be provided. 

**NOTE**: All the input information should be in S.I units, e.g: ionic strength, metabolite concentration in Moles.

To demonstrate the usage, we use *E. Coli* core model from `BIGG database`.


In [7]:
from cobra import io
from tmfa.core import tmodel

# Load COBRA model
cobra_model = io.load_matlab_model('e_coli_core.mat')

# compartment specific properties
pH_I_dict = {
    "pH": {"c": 7.5, "e": 7, "p": 7},
    "I": {"c": 0.25, "e": 0, "p": 0},
}

# Membrane potentials
del_psi_dict = { 
"c": {"c":0,"e": 0, "p": 150}, 
"e": {"c": 0,"e":0, "p": 0}, 
"p": {"c": -150, "e": 0, "p":0}, 
} 

import pandas as pd
# Define membrane potential and compartment information in Dataframes
del_psi = pd.DataFrame.from_dict(data=del_psi_dict)
comp_info = pd.DataFrame.from_dict(data=pH_I_dict)

# List of reaction ids that you would like to exclude from thermodynamic analysis, here we exclude demand reactions
Excl = [
    rxn.id
    for rxn in cobra_model.reactions
    if rxn.id.startswith("EX_") or rxn.id.startswith("DM_")
] + ["BIOMASS_Ecoli_core_w_GAM", "O2t", "H2Ot"]


# Now build the multiTFA model from COBRA model
tfa_model = tmodel(
    cobra_model, Exclude_list=Excl, compartment_info=comp_info, membrane_potential=del_psi
)


In order to access the thermodynamic information from equilibrator api, we need to add a external database identifier against all the metabolites in tfa model. This can be done by populating the 'Kegg_id' property of the metabolites. If you do not know the identifier for some metabolites, it is okay, you can simply add `NA` against it. For simplicity, we use the notation similar to equilibrator api. For the example below in the *E. coli* core model, we use bigg identifiers. You can follow equilibrator api notation for other database identifiers, for example, if you are using `KEGG` identifier, simply use `kegg:kegg identifier`.

In [14]:
for met in tfa_model.metabolites: 
    kegg_id = 'bigg.metabolite:'+met.id[:-2] 
    met.Kegg_id = kegg_id

tfa_model.update()

## 1.2 Metabolites

Reactions and metabolites in multiTFA model can be accessed similar to a COBRA model. For demonstration, we will consider cytosolic `ATP` as our metabolite with `atp_c` identifier in tfa_model. We will access `ATP` and print it's standard Gibbs energy of formation and Also, change its intracellular concentration lower and upper bounds to '2e-3' and '5e-2' M respectively from it's default bounds. These changes will reflect in solver variables and constraints.

In [15]:
atp = tfa_model.metabolites.get_by_id('atp_c')
print("Standard Gibbs energy of formation of ATP is {} KJ/mol".format(atp.delG_f))
print("initial concentrations")
print('{}\t{}\t{}'.format(atp.id, atp.concentration_min, atp.concentration_max))
atp.concentration_min = 2e-3
atp.concentration_max = 5e-2
print("Concentrations after modifying")
print('{}\t{}\t{}'.format(atp.id, atp.concentration_min, atp.concentration_max))

Standard Gibbs energy of formation of ATP is -2259.1882733696866 KJ/mol
initial concentrations
atp_c	0.002	0.02
Concentrations after modifying
atp_c	0.002	0.05


You can always change the Gibbs energy of formation value of a metabolite by resetting the `delG_f` property of the metabolite. Also, you can access additional information like, major pseudoisomer at given compartment conditions, equilibrator accession of the metabolite.

## 1.3 Reactions

Reactions in multiTFA model are of type cobra.DictList. Similar to COBRA model, reactions can be accessed/retrieved by its index or by it's name and properties/attributes can be accessed. Let us consider ATP synthase raction in our model by 'ATPS4rpp' and access it's attributes.


In [18]:
atps = tfa_model.reactions.get_by_id('ATPS4r')
print(atps.reaction)

adp_c + 4.0 h_e + pi_c <=> atp_c + h2o_c + 3.0 h_c


We can access the Gibbs energy of transport and transformed Gibbs energy of the reaction

In [20]:
print("Gibbs energy of transport is {} KJ/mol".format(atps.delG_transport))
print("Transform Gibbs energy of is {} KJ/mol".format(atps.delG_prime))

Gibbs energy of transport is -4.605170185988092 KJ/mol
Transform Gibbs energy of is 32.396004094027376 KJ/mol


You can always reset the transport and transformed Gibbs energy of reactions. You can also access the stoichiometric matrix of the reaction, all the thermodynamic constraints associated with the reaction.